In [1]:
from scipy.sparse import csr_matrix, coo_matrix, lil_matrix, dok_matrix
import scipy.sparse
import numpy as np
from scipy.sparse.linalg import eigsh
from networkx.generators.community import planted_partition_graph
import networkx as nx
from sklearn.cluster import KMeans

In [2]:
N = 1 * 10**5
q = 2

average_degree = 5.0 # average degree
diff = 7.5 # cin - cout
cout = average_degree * 2 - diff
cin = average_degree * 2 - cout
seed = 13
print(cin-cout, 2*np.sqrt(average_degree), (cin-cout)/(2*np.sqrt(average_degree)))

G = planted_partition_graph(q, int(N/q), cin / N, cout / N, seed=seed)
partition = np.zeros(N)
for i, part in enumerate(G.graph['partition']):
    partition[list(part)] = i
    
A = nx.adjacency_matrix(G)

5.0 4.47213595499958 1.118033988749895


In [3]:
from matplotlib.pyplot import figure
%matplotlib inline

def plot_graph(G, subset = None):
    if not subset:
        subset = {}
        
    figure(figsize=(10, 10/1.33))
    node_types = [1 if node in subset else 0 for node in G.nodes()]
    rng = np.random.RandomState(1)
    pos = {node: rng.rand(2) for node in G.nodes()}
    nx.draw_networkx(
        G, 
        node_color=node_types, 
        with_labels=False, 
        pos=nx.spring_layout(G, pos = pos, k=7.00/np.sqrt(len(G)))
    )
    
#plot_graph(G)

In [4]:
kmeans = KMeans(n_clusters=2, random_state=0)
L = nx.normalized_laplacian_matrix(G)
vals, vecs = scipy.sparse.linalg.eigsh(L, which="SA", k=3)
kmeans.fit(vecs[:,1:])
labels = kmeans.labels_
#labels = (vecs[:,0] <= 0).astype(np.int)
size_of_partition = np.sum(labels == 0) / vecs.shape[0]
partitioning = max(size_of_partition, 1-size_of_partition)
accuracy = (partition == labels).mean()
print(f"{max(accuracy, 1-accuracy):.2f}, {partitioning:.2f}")

#subset = set(np.argwhere(labels).flatten().tolist())
#plot_graph(G, subset)

0.50, 1.00


In [5]:
def build_unweighted_bethe_hessian(adjacency_matrix):
    n = adjacency_matrix.shape[0]
    diags = np.asarray(adjacency_matrix.sum(axis=1).flatten())
    D = scipy.sparse.spdiags(diags, [0], n, n, format='csr')
    I = scipy.sparse.eye(n, format='csr')
    r = np.sqrt(np.mean(diags**2) / np.mean(diags) - 1)
    Hr = (r ** 2 - 1) * I - r * adjacency_matrix + D
    return Hr

In [6]:
bethe_hessian = build_unweighted_bethe_hessian(A)

In [7]:
vals, vecs = eigsh(bethe_hessian, 2, which='SA')

In [8]:
accuracy = (partition == (vecs[:,1] <= 0).astype(np.int)).mean()
max(accuracy, 1-accuracy)

0.70256

In [9]:
kmeans = KMeans(n_clusters=2, random_state=0)
kmeans.fit(vecs[:,1:])
accuracy = (partition == kmeans.labels_).mean()
max(accuracy, 1-accuracy)

0.7025

In [10]:
#subset = set(np.argwhere(vecs[:,1] <= 0).flatten().tolist())
#plot_graph(G, subset)

In [12]:
import scipy

def build_pmi_matrix(adjacency_matrix, neg):
    n = adjacency_matrix.shape[0]
    diags = np.asarray(adjacency_matrix.sum(axis=1)).flatten()
    with scipy.errstate(divide='ignore'):
        diags_inv = 1.0 / diags
    diags_inv[scipy.isinf(diags_inv)] = 0
    D_inv = scipy.sparse.spdiags(diags_inv, [0], n, n, format='csr')
    pmi = D_inv.dot(adjacency_matrix.dot(D_inv))
    #dataset_size = adjacency_matrix.sum()
    #print(adjacency_matrix.sum())
    dataset_size = 1.0
    pmi.data = np.log(pmi.data * dataset_size / neg)
    
    with scipy.errstate(divide='ignore'):
        diags_inv = np.log((1.0 / diags)*dataset_size / neg)
    diags_inv[scipy.isinf(diags_inv)] = 0
    D_inv = scipy.sparse.spdiags(diags_inv, [0], n, n, format='csr')
    
    #pmi.data[pmi.data < 0] = 0
    return pmi

def build_tfidf_matrix(adjacency_matrix, neg):
    n = adjacency_matrix.shape[0]
    diags = np.asarray(adjacency_matrix.sum(axis=1)).flatten()
    total_sum = adjacency_matrix.sum()
    with scipy.errstate(divide='ignore'):
        diags_inv = np.log(total_sum / diags)
    diags_inv[scipy.isinf(diags_inv)] = 0
    D_inv = scipy.sparse.spdiags(diags_inv, [0], n, n, format='csr')
    tfidf = adjacency_matrix.dot(D_inv)
    return tfidf

def pmi_laplacian(pmi):
    n = pmi.shape[0]
    diags = np.asarray(pmi.sum(axis=1).flatten())
    D = scipy.sparse.spdiags(diags, [0], n, n, format='csr')
    return D - pmi

In [14]:
from sklearn.cluster import KMeans
from scipy.sparse.linalg import eigsh, svds

kmeans = KMeans(n_clusters=2, random_state=0)

#for k in range(10, 100, 10):
k=3
tfidf = build_tfidf_matrix(A.astype(np.float64), 1)
#pmi = build_pmi_matrix(A.astype(np.float64), k / 25000)
#pmi = build_pmi_matrix(A.astype(np.float64), 1)
#pmi = pmi_laplacian(pmi)
vecs, vals, vecs2 = svds(tfidf, 3, which='LM')
# vals, vecs = eigsh(pmi, 3, which='LM')
#print(vals)
kmeans.fit(vecs)
labels = kmeans.labels_
#labels = (vecs[:,0] <= 0).astype(np.int)
size_of_partition = np.sum(labels == 0) / vecs.shape[0]
partitioning = max(size_of_partition, 1-size_of_partition)
accuracy = (partition == labels).mean()
print(f"{k}, {np.log(k):.2f}, {max(accuracy, 1-accuracy):.3f}, {partitioning:.2f}")

#subset = set(np.argwhere(labels).flatten().tolist())
#plot_graph(G, subset)

3, 1.10, 0.500, 1.00


In [21]:
tfidf.data.min()

10.232091614508498

In [93]:
pmi[:10,:10].todense()

matrix([[-18.46160273,   0.        ,   0.        ,   0.        ,
           0.        ,   0.        ,   0.        ,   0.        ,
           0.        ,   0.        ],
        [  0.        , -17.95077711,   0.        ,   0.        ,
           0.        ,   0.        ,   0.        ,   0.        ,
           0.        ,   0.        ],
        [  0.        ,   0.        , -17.95077711,   0.        ,
           0.        ,   0.        ,   0.        ,   0.        ,
           0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        , -17.76845555,
           0.        ,   0.        ,   0.        ,   0.        ,
           0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        ,   0.        ,
         -17.76845555,   0.        ,   0.        ,   0.        ,
           0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        ,   0.        ,
           0.        , -18.46160273,   0.        ,   0.        ,
           0.        ,   0.   

In [483]:
#pmi = build_pmi_matrix(A.astype(np.float64), 1.0)
pmi.eliminate_zeros()

In [485]:
pmi.data.max()

6.437751649736401